In [24]:
import pandas as pd
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import joblib
df = pd.read_csv("miss_smaler_7per.csv")


In [25]:
df = df[['home_batting_batting_avg_10RA',
         'away_batting_batting_avg_10RA',
        'date',
        "home_pitching_earned_run_avg_10RA",
        'away_pitching_earned_run_avg_10RA',
        'home_batting_onbase_plus_slugging_10RA',
        'away_batting_onbase_plus_slugging_10RA',
        'home_batting_RBI_10RA',
        'away_batting_RBI_10RA',
        'home_team_wins_mean' ,
        'away_team_wins_mean' ,
        #'home_team_errors_mean',
        #'away_team_errors_mean',
        'home_team_abbr' , 
        'away_team_abbr',
        'home_team_win' ,
        'home_pitching_SO_batters_faced_mean',
        'away_pitching_SO_batters_faced_mean',
        'home_pitching_wpa_def_mean',
         'away_pitching_wpa_def_mean',
        'home_batting_wpa_bat_mean',
        'away_batting_wpa_bat_mean' ,
        'home_team_spread_mean' ,
        'away_team_spread_mean' 




        ]]
 
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year


df.drop(columns=['date'], inplace=True, errors='ignore')
df

,home_batting_batting_avg_10RA,away_batting_batting_avg_10RA,home_pitching_earned_run_avg_10RA,away_pitching_earned_run_avg_10RA,home_batting_onbase_plus_slugging_10RA,away_batting_onbase_plus_slugging_10RA,home_batting_RBI_10RA,away_batting_RBI_10RA,home_team_wins_mean,away_team_wins_mean,...,home_team_win,home_pitching_SO_batters_faced_mean,away_pitching_SO_batters_faced_mean,home_pitching_wpa_def_mean,away_pitching_wpa_def_mean,home_batting_wpa_bat_mean,away_batting_wpa_bat_mean,home_team_spread_mean,away_team_spread_mean,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11062,-0.812161,-0.478545,0.365706,0.566667,-0.296017,-0.333902,-0.472473,-0.647647,0.199443,0.791356,...,False,0.295225,0.768515,0.586711,0.379027,-0.281820,0.832028,0.066197,1.122813,2023
11063,0.280776,0.478694,-0.222728,-1.046506,0.154042,0.560315,-0.294963,0.327235,-0.966669,-0.093481,...,True,-1.619651,0.427738,-1.101072,0.192703,-0.360312,-0.352853,-1.309128,0.529167,2023
11064,-1.639621,-0.089452,0.168184,NaN,-1.369143,0.093464,-1.537530,-0.736273,0.403748,0.233788,...,False,0.100324,-0.262025,0.501567,-0.550230,0.107963,0.959052,0.195615,-0.017500,2023
11065,0.973774,-1.697474,-1.077280,-0.381872,1.353418,-1.586968,0.503830,-0.559022,0.486256,0.938194,...,False,1.125557,0.496456,0.911125,NaN,-0.172485,1.017721,0.447259,0.404946,2023


In [26]:
df['diff_batting_avg_10RA'] = df['home_batting_batting_avg_10RA'] - df['away_batting_batting_avg_10RA']
df['diff_ERA_10RA'] = df['home_pitching_earned_run_avg_10RA'] - df['away_pitching_earned_run_avg_10RA']
df['diff_batting_onbase_plus_slugging_10RA'] = df['home_batting_onbase_plus_slugging_10RA'] - df['away_batting_onbase_plus_slugging_10RA']
df['diff_batting_RBI_10RA'] = df['home_batting_RBI_10RA'] - df['away_batting_RBI_10RA']
df['diff_home_pitching_SO_batters_faced_mean'] = df['home_pitching_SO_batters_faced_mean'] - df['away_pitching_SO_batters_faced_mean']
df ['plus_home_pitching_wpa_def_batted_mean']  = df['home_batting_wpa_bat_mean'] +  df['home_pitching_wpa_def_mean']
df ['plus_away_pitching_wpa_def_batted_mean']  = df['away_batting_wpa_bat_mean'] + df['away_pitching_wpa_def_mean']
df['diff_plus_pitching_wpa_def_batted_mean'] = df['plus_home_pitching_wpa_def_batted_mean']-df['plus_away_pitching_wpa_def_batted_mean']
df['spread_team_mean']  = df['home_team_spread_mean'] / (df['away_team_spread_mean'] + 1)



columns_to_drop = [
    'home_batting_batting_avg_10RA', 'away_batting_batting_avg_10RA', 
    'home_batting_onbase_perc_10RA', 'away_batting_onbase_perc_10RA',
    'home_batting_onbase_plus_slugging_10RA','away_batting_onbase_plus_slugging_10RA',
    'home_batting_RBI_10RA','away_batting_RBI_10RA',   
    'home_pitching_earned_run_avg_10RA','away_pitching_earned_run_avg_10RA',
    'home_pitching_wpa_def_mean' , 'away_pitching_wpa_def_mean',
    'home_batting_wpa_bat_mean' , 'away_batting_wpa_bat_mean',
    "home_pitching_SO_batters_faced_mean","away_pitching_SO_batters_faced_mean",
    "plus_home_pitching_wpa_def_batted_mean","plus_away_pitching_wpa_def_batted_mean",
    'home_team_spread_mean','away_team_spread_mean'
 ]

df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df

,home_team_wins_mean,away_team_wins_mean,home_team_abbr,away_team_abbr,home_team_win,year,diff_batting_avg_10RA,diff_ERA_10RA,diff_batting_onbase_plus_slugging_10RA,diff_batting_RBI_10RA,diff_home_pitching_SO_batters_faced_mean,diff_plus_pitching_wpa_def_batted_mean,spread_team_mean
0,NaN,NaN,KFH,SAJ,False,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,JBM,KJP,False,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,VQC,BPH,True,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,VJV,MOO,False,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,PDF,HAN,False,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11062,0.199443,0.791356,UPV,KFH,False,2023,-0.333616,-0.200962,0.037885,0.175174,-0.473289,-0.906163,0.031184
11063,-0.966669,-0.093481,GKO,YHA,True,2023,-0.197917,0.823777,-0.406274,-0.622198,-2.047389,-1.301234,-0.856106
11064,0.403748,0.233788,QPO,XFB,False,2023,-1.550169,NaN,-1.462607,-0.801257,0.362349,0.200708,0.199099
11065,0.486256,0.938194,SAJ,STC,False,2023,2.671249,-0.695408,2.940386,1.062851,0.629101,NaN,0.318346


In [27]:
# 分離目標變量
y = df['home_team_win'].astype(bool)
X = df.drop(columns=['home_team_win'])

# 編碼分類特徵（隊伍和日期相關）
cat_features = ['home_team_abbr', 'away_team_abbr']
num_features = [col for col in X.columns if col not in cat_features + ['year']]

# 定義預處理器
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
        ('num', 'passthrough', num_features)
    ]
)

pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('imputer', SimpleImputer(strategy='mean')),
    ('clf', SVC(probability=True, random_state=42))  # 使用 SVM
])
 
param_grid = {
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],  
    'clf__gamma': ['scale', 'auto', 0.0001, 0.001, 0.01, 0.1, 1],  
    'clf__class_weight': [None, 'balanced']  
}
grid = list(ParameterGrid(param_grid))

In [28]:
available_years = sorted(df['year'].unique())
print(f"可用的年份: {available_years}")

models = {}

def split_train_test_per_year(df, year, test_size=0.2):
    df_year = df[df['year'] == year]
    n_total = len(df_year)
    if n_total == 0:
        print(f"警告：{year}年沒有數據。")
        return pd.DataFrame(), pd.DataFrame()
    n_test = max(1, int(n_total * test_size))   
    df_train = df_year.iloc[:-n_test]
    df_test = df_year.iloc[-n_test:]
    return df_train, df_test


可用的年份: [2016, 2017, 2018, 2019, 2021, 2022, 2023]


In [29]:
for year in available_years:
    print(f"\n訓練 {year} 年的模型...")
    train_df, test_df = split_train_test_per_year(df, year, test_size=0.2)
    
    if train_df.empty:
        print(f"警告：{year}年的訓練集為空，跳過該年份的模型訓練。")
        continue
    
    X_train = train_df.drop(columns=['home_team_win', 'year'])
    y_train = train_df['home_team_win']
    
    X_test = test_df.drop(columns=['home_team_win', 'year'])
    y_test = test_df['home_team_win']
    
    best_score = -np.inf
    best_params = None
    
    for params in grid:
        pipeline.set_params(**params)
        
        try:
            pipeline.fit(X_train, y_train)
        except Exception as e:
            print(f"錯誤：{year}年使用參數 {params} 訓練失敗。錯誤信息：{e}")
            continue
        
        try:
            # 在測試集上評估
            y_pred = pipeline.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
        except Exception as e:
            print(f"錯誤：{year}年模型在測試集評估時失敗。錯誤信息：{e}")
            continue
        
        # 更新最佳參數
        if acc > best_score:
            best_score = acc
            best_params = params
    
    print(f"最佳參數: {best_params}")
    print(f"驗證最佳準確率: {best_score:.4f}")
    
    pipeline.set_params(**best_params)
    try:
        pipeline.fit(X_train, y_train)
    except Exception as e:
        print(f"錯誤：{year}年使用最佳參數 {best_params} 訓練失敗。錯誤信息：{e}")
        continue
    
    models[year] = pipeline


print("\n所有年份的模型訓練和評估完成。")

joblib.dump(models, 'trained_models_3.pkl')
print("模型已保存至 'trained_models.pkl'。")


訓練 2016 年的模型...
最佳參數: {'clf__C': 0.001, 'clf__class_weight': None, 'clf__gamma': 'scale'}
驗證最佳準確率: 0.5751

訓練 2017 年的模型...
最佳參數: {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__gamma': 0.001}
驗證最佳準確率: 0.5732

訓練 2018 年的模型...
最佳參數: {'clf__C': 10, 'clf__class_weight': None, 'clf__gamma': 0.0001}
驗證最佳準確率: 0.5763

訓練 2019 年的模型...
最佳參數: {'clf__C': 0.1, 'clf__class_weight': 'balanced', 'clf__gamma': 0.1}
驗證最佳準確率: 0.5957

訓練 2021 年的模型...
最佳參數: {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__gamma': 1}
驗證最佳準確率: 0.5559

訓練 2022 年的模型...
最佳參數: {'clf__C': 1, 'clf__class_weight': None, 'clf__gamma': 'auto'}
驗證最佳準確率: 0.6144

訓練 2023 年的模型...
最佳參數: {'clf__C': 0.1, 'clf__class_weight': 'balanced', 'clf__gamma': 'auto'}
驗證最佳準確率: 0.5705

所有年份的模型訓練和評估完成。
模型已保存至 'trained_models.pkl'。


In [30]:
#test
test_df1 = pd.read_csv("test_2.csv")

test_df = test_df1[['home_batting_batting_avg_10RA','id',
                    
         'away_batting_batting_avg_10RA',
        "home_pitching_earned_run_avg_10RA",
        'away_pitching_earned_run_avg_10RA',
        'home_batting_onbase_plus_slugging_10RA',
        'away_batting_onbase_plus_slugging_10RA',
        'home_batting_RBI_10RA',
        'away_batting_RBI_10RA',
        'home_team_wins_mean' ,
        'away_team_wins_mean' ,
        #'home_team_errors_mean',
        #'away_team_errors_mean',
        'home_team_abbr' , 
        'away_team_abbr',
        'home_pitching_SO_batters_faced_mean',
        'away_pitching_SO_batters_faced_mean',
        'home_pitching_wpa_def_mean',
        'away_pitching_wpa_def_mean',
        'home_team_spread_mean',
        'away_team_spread_mean',
        'home_batting_wpa_bat_mean',
        'away_batting_wpa_bat_mean' ,





        ]]

test_df['year'] = test_df1['season']

test_df['diff_batting_avg_10RA'] = test_df['home_batting_batting_avg_10RA'] - test_df['away_batting_batting_avg_10RA']
test_df['diff_ERA_10RA'] = test_df['home_pitching_earned_run_avg_10RA'] - test_df['away_pitching_earned_run_avg_10RA']
test_df['diff_batting_onbase_plus_slugging_10RA'] = test_df['home_batting_onbase_plus_slugging_10RA'] - test_df['away_batting_onbase_plus_slugging_10RA']
test_df['diff_batting_RBI_10RA'] = test_df['home_batting_RBI_10RA'] - test_df['away_batting_RBI_10RA']
test_df['diff_home_pitching_SO_batters_faced_mean'] = test_df['home_pitching_SO_batters_faced_mean'] - test_df['away_pitching_SO_batters_faced_mean']
test_df ['plus_home_pitching_wpa_def_batted_mean']  = test_df['home_batting_wpa_bat_mean'] +  test_df['home_pitching_wpa_def_mean']
test_df ['plus_away_pitching_wpa_def_batted_mean']  = test_df['away_batting_wpa_bat_mean'] + test_df['away_pitching_wpa_def_mean']
test_df['diff_plus_pitching_wpa_def_batted_mean'] = test_df['plus_home_pitching_wpa_def_batted_mean']-test_df['plus_away_pitching_wpa_def_batted_mean']
test_df['spread_team_mean']  = test_df['home_team_spread_mean'] / (test_df['away_team_spread_mean'] + 1)



test_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_df


C:\Users\cheeh\AppData\Local\Temp\ipykernel_6936\4078011199.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['year'] = test_df1['season']
C:\Users\cheeh\AppData\Local\Temp\ipykernel_6936\4078011199.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['diff_batting_avg_10RA'] = test_df['home_batting_batting_avg_10RA'] - test_df['away_batting_batting_avg_10RA']
C:\Users\cheeh\AppData\Local\Temp\ipykernel_6936\4078011199.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,id,home_team_wins_mean,away_team_wins_mean,home_team_abbr,away_team_abbr,year,diff_batting_avg_10RA,diff_ERA_10RA,diff_batting_onbase_plus_slugging_10RA,diff_batting_RBI_10RA,diff_home_pitching_SO_batters_faced_mean,diff_plus_pitching_wpa_def_batted_mean,spread_team_mean
0,0,1.183823,-0.248190,HXK,SAJ,2017,0.391577,0.401906,-0.446938,-0.620906,1.601762,2.137409,4.945829
1,1,-0.457296,0.519884,VJV,DPS,2023,-0.918986,-0.157669,-1.352404,-2.040983,0.267404,NaN,-0.279268
2,2,0.311179,-0.011664,BPH,KJP,2016,-0.407589,-1.361911,NaN,NaN,-1.009788,NaN,NaN
3,3,-1.781596,-0.476268,GLO,PJT,2019,0.629201,0.124329,1.202063,0.441697,0.123148,-1.945597,-33.670627
4,4,NaN,0.843798,PDF,HAN,2016,1.329490,2.926144,0.912330,0.353588,-1.993361,NaN,-0.514559
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6180,6180,-0.008791,-1.124522,PJT,GLO,2022,1.892230,-0.138062,1.917314,0.440534,1.431323,1.684117,0.946958
6181,6181,1.586734,-0.277152,RLJ,KJP,2018,-0.039596,0.171108,-1.504998,NaN,0.773230,2.805573,1.568843
6182,6182,0.080452,-0.100160,VJV,DPS,2017,NaN,NaN,-0.375928,-0.621552,1.425247,0.243717,0.039633
6183,6183,0.416354,0.556542,KFH,FBW,2018,1.483972,NaN,1.823945,1.063884,-0.783519,-0.221076,0.200773


In [31]:

unique_years = test_df['year'].unique()

for yr in unique_years:
    if yr not in models:
        print(f"警告：{yr}年的模型不存在，該年份資料將跳過預測。")
        continue

    df_year = test_df[test_df['year'] == yr].copy()
 
    df_year.drop(columns=['year', 'date'], errors='ignore', inplace=True)
    
    y_pred = models[yr].predict(df_year)
    
    # 將預測結果寫回 test_df 對應的列
    test_df.loc[df_year.index, 'home_team_win'] = y_pred


submission = test_df[['id', 'home_team_win']].copy()
submission.to_csv("submission_3_1.5.csv", index=False)
print("submission.csv 已建立完成。")


C:\Users\cheeh\AppData\Local\Temp\ipykernel_6936\3630104581.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[df_year.index, 'home_team_win'] = y_pred


submission.csv 已建立完成。
